In [4]:
import pandas as pd
clean_frame_pred = pd.read_csv("cleaned_dataset.csv")
merged_data = pd.read_csv('merged.csv')

In [7]:
merged_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'quadkey', 'avg_d_kbps', 'avg_u_kbps',
       'avg_lat_ms', 'tests', 'devices', 'year', 'quarter', 'conn_type',
       'PRUID', 'PRNAME', 'CDUID', 'CDNAME', 'DAUID', 'SACTYPE', 'DA_POP',
       'PCUID', 'PCNAME', 'PCTYPE', 'PCCLASS', 'geometry', 'd_score',
       'u_score', 'closest_city', 'closest_coord', 'distance_to_closest_city',
       'closest_PC_name', 'closest_PC_dist'],
      dtype='object')

In [3]:
clean_frame_pred.columns

Index(['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'year',
       'quarter', 'conn_type', 'PRUID', 'PRNAME', 'CDUID', 'CDNAME', 'DAUID',
       'SACTYPE', 'DA_POP', 'PCTYPE', 'd_score', 'u_score', 'closest_city',
       'distance_to_closest_city', 'closest_PC_name', 'closest_PC_dist', 'x',
       'y', 'Y'],
      dtype='object')

In [6]:
import numpy as np
clean_frame_pred['total'] = np.ones(len(clean_frame_pred['y']))

In [7]:
clean_frame_pred.isnull().values.any()

False

In [10]:
sub_pred = clean_frame_pred[['DAUID','conn_type','x','y','Y','distance_to_closest_city','closest_PC_dist',
                             'DA_POP','total','PRNAME','year','quarter']]
f = {'x':['mean'], 'y':['mean'],'Y':['sum'],
     'distance_to_closest_city':['mean'],'closest_PC_dist':['mean'],'DA_POP':['mean'],'total':['sum']}
grouped_df = sub_pred.groupby(by = ['DAUID','PRNAME','year','quarter']).agg(f)
grouped_df

x  \
                                                                                  mean   
DAUID    PRNAME                                             year quarter                 
10010165 Newfoundland and Labrador / Terre-Neuve-et-Labr... 2019 Q1       8.978154e+06   
                                                                 Q2       8.978154e+06   
                                                                 Q3       8.978223e+06   
                                                                 Q4       8.978154e+06   
                                                            2020 Q1       8.978154e+06   
...                                                                                ...   
61060164 Northwest Territories / Territoires du Nord-Ouest  2021 Q4       5.093506e+06   
62040060 Nunavut                                            2019 Q3       7.292991e+06   
                                                            2021 Q1       7.292991e+06   
                                                                 Q2       7.292991e+06   
62050020 Nunavut                                            2019 Q2       6.188403e+06   

                                                                                     y  \
                                                                                  mean   
DAUID    PRNAME                                             year quarter                 
10010165 Newfoundland and Labrador / Terre-Neuve-et-Labr... 2019 Q1       2.146515e+06   
                                                                 Q2       2.146515e+06   
                                                                 Q3       2.146479e+06   
                                                                 Q4       2.146515e+06   
                                                            2020 Q1       2.146515e+06   
...                                                                                ...   
61060164 Northwest Territories / Territoires du Nord-Ouest  2021 Q4       3.095757e+06   
62040060 Nunavut                                            2019 Q3       3.241209e+06   
                                                            2021 Q1       3.241209e+06   
                                                                 Q2       3.241209e+06   
62050020 Nunavut                                            2019 Q2       2.937126e+06   

                                                                           Y  \
                                                                         sum   
DAUID    PRNAME                                             year quarter       
10010165 Newfoundland and Labrador / Terre-Neuve-et-Labr... 2019 Q1        2   
                                                                 Q2        3   
                                                                 Q3        4   
                                                                 Q4        3   
                                                            2020 Q1        3   
...                                                                       ..   
61060164 Northwest Territories / Territoires du Nord-Ouest  2021 Q4        1   
62040060 Nunavut                                            2019 Q3        0   
                                                            2021 Q1        0   
                                                                 Q2        1   
62050020 Nunavut                                            2019 Q2        0   

                                                                         distance_to_closest_city  \
                                                                                             mean   
DAUID    PRNAME                                             year quarter                            
10010165 Newfoundland and Labrador / Terre-Neuve-et-Labr... 2019 Q1                  8.078236e+05   
                                      

In [11]:
grouped_df.columns = ['x','y','Y','distance_to_closest_city','closest_PC_dist','DA_POP','total']

In [13]:
mode_df= sub_pred.groupby(['DAUID','PRNAME','year','quarter'])['conn_type'].agg(pd.Series.mode).to_frame()

In [15]:
final_clean_data_pred  = grouped_df.join(mode_df, on = ['DAUID','PRNAME','year','quarter'])

In [16]:
clean_frame_pred.isnull().values.any()

False

In [22]:
final_clean_data_pred = final_clean_data_pred.reset_index(level=[1,2,3])

In [31]:
new_feature = np.zeros(len(final_clean_data_pred['x']))
years = final_clean_data_pred['year']
quarters = final_clean_data_pred['quarter']
for i in range(0,len(quarters)):
    new_feature[i] = 4*(float(years.iloc[i]) - 2019) + float(quarters.iloc[i][1])
final_clean_data_pred['time'] = new_feature

In [39]:
final_clean_data_pred = final_clean_data_pred.drop(columns = ['year','quarter'])
final_quarter_data = final_clean_data_pred.loc[final_clean_data_pred['time'] == 12]
est_data = final_clean_data_pred.loc[final_clean_data_pred['time'] != 12]

In [41]:
est_data.to_csv('estimation_data_full.csv')

In [42]:
final_quarter_data.to_csv('final_quarter_data.csv')